In [95]:
import requests
import telepot
import datetime
import pandas as pd
import numpy as np
import json
from pandas import json_normalize

import selenium
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import datetime
from selenium.webdriver.common.action_chains import ActionChains

In [96]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", user_agent)
#帳號密碼
username="bbtorin" 
passwd="qwe123"
nowtime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [97]:
###################  登入系統  ################
fp = webdriver.FirefoxProfile()
fp.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory 
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain,text/x-csv,text/csv,application/vnd.ms-excel,application/csv,application/x-csv,text/csv,text/comma-separated-values,text/x-comma-separated-values,text/tab-separated-values,application/pdf")
fp.set_preference("browser.download.manager.showWhenStarting",False)
fp.set_preference("browser.helperApps.neverAsk.openFile","text/plain,text/x-csv,text/csv,application/vnd.ms-excel,application/csv,application/x-csv,text/csv,text/comma-separated-values,text/x-comma-separated-values,text/tab-separated-values,application/pdf")
fp.set_preference("browser.helperApps.alwaysAsk.force", False)
fp.set_preference("browser.download.manager.useWindow", False)
fp.set_preference("browser.download.manager.focusWhenStarting", False)
fp.set_preference("browser.helperApps.neverAsk.openFile", "")
fp.set_preference("browser.download.manager.alertOnEXEOpen", False)
fp.set_preference("browser.download.manager.showAlertOnComplete", False)
fp.set_preference("browser.download.manager.closeWhenDone", True)
fp.set_preference("browser.download.dir", "F:\Desktop\download_csv") 
options = Options()
options.add_argument('--headless')
options.binary = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\firefox.exe')
browser = webdriver.Firefox(executable_path=r'F:\Desktop\python_code\geckodriver.exe', options=options,firefox_profile = fp)
browser.maximize_window()
browser.get('https://yb01.88lard.com/')
time.sleep(3)

#elem=browser.find_element_by_id("username")
elem=browser.find_element_by_name("username")
elem.send_keys(username)
#elem=browser.find_element_by_id("password")
elem=browser.find_element_by_name("password")
elem.send_keys(passwd)
elem=browser.find_element_by_class_name("ui.large.fluid.button.submit")
elem.click()
time.sleep(3)

In [98]:
#點選進入迅付
browser.find_element_by_class_name('item.sidebar-tab.extension-menu').click()
time.sleep(1)
browser.find_element_by_xpath('//div[@class="sidebar-text"][text()="迅付"]').click() 
time.sleep(1) 

In [99]:
#換分頁
browser1=browser.window_handles[1]
time.sleep(1)
browser.switch_to_window(browser1) 
time.sleep(1)

<ipython-input-99-f34807bf05b7>:4: DeprecationWarning:

use driver.switch_to.window instead



In [100]:
#抵達會員入款訊息 
time.sleep(5)
browser.find_element_by_xpath('//*[@id="site-container"]/nav/div[2]/div[1]/div[2]/div/ul/li[5]/div/div[1]/a').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="site-content"]/div/div[1]/button[3]').click()

In [101]:
cookies = browser.get_cookies()  # 获取浏览器cookies
payid=cookies[0]['value']

In [102]:
#登入訊息
headers = {
'cookie': 'lang=zh-cn; payid='+payid,
#'referer': weblink,
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
} 


#美東時間
aaaaa1=(datetime.datetime.now() - datetime.timedelta(hours=36)).strftime("%Y-%m-%d") 
aaaaa2=(datetime.datetime.now() - datetime.timedelta(hours=12)).strftime("%Y-%m-%d") 

In [103]:
#抓取更動
session_requests = requests.session()
ttt = session_requests.get("https://a.inpay-pro.com/api/trade/v1/monitor/account_log?created_at_start="+aaaaa1+"T00%3A00%3A00%2B0800&created_at_end="+aaaaa2+"T11%3A59%3A59%2B0800&vendor_id=81&purpose=payment&first_result=0&max_results=200&sort=created_at&order=desc" , headers=headers )
#抓取上限下限                  
ttt222 = session_requests.get("https://a.inpay-pro.com/api/trade/v1/merchant/deposit/list?first_result=0&max_results=200&sort=id&order=asc" , headers=headers )

In [104]:
list_data_all_f=[]
for i in range(0, 10000, 1000) :
    ttt333 = session_requests.get("https://a.inpay-pro.com/api/trade/v1/deposit/entry/list?created_at_start="+aaaaa1+"T00%3A00%3A00%2B0800&created_at_end="+aaaaa2+"T11%3A59%3A59%2B0800&sort=id&order=desc&first_result="+str(i)+"&max_results=1000" , headers=headers )
    output_data=ttt333.json()['ret']
    list_data_all_f=list_data_all_f+output_data

In [105]:
list_data_all_s=[]
for i in range(0, 10000, 1000) :   
    ttt444 = session_requests.get("https://a.inpay-pro.com/api/trade/v1/deposit/entry/list?state_at_start="+aaaaa1+"T00%3A00%3A00%2B0800&state_at_end="+aaaaa2+"T11%3A59%3A59%2B0800&sort=id&order=desc&first_result="+str(i)+"&max_results=1000" , headers=headers )
    output_data_s=ttt444.json()['ret']
    list_data_all_s=list_data_all_s+output_data_s

In [106]:
all_channel=pd.DataFrame(list_data_all_s)[['merchant_alias','bank_name']]
all_channel=all_channel[all_channel['merchant_alias'] != '代客充值-鸭脖银行卡支援']

In [107]:
all_channel=pd.DataFrame(all_channel.groupby(['merchant_alias', 'bank_name']).size().reset_index().rename(columns={"size": "total_count"}))
all_channel.columns = ['Resource','bank_name','total_count']
all_channel=all_channel[['Resource','bank_name']]

In [108]:
browser.close()
time.sleep(1)
browser.switch_to_window(browser.window_handles[0]) 
time.sleep(1)
browser.close()

<ipython-input-108-967e24157dae>:3: DeprecationWarning:

use driver.switch_to.window instead



In [109]:
df_data=pd.DataFrame(ttt.json()['ret'])
df_data=df_data[['alias','status','created_at']]

In [110]:
name_list=np.unique(df_data['alias'])
name_list

array(['CGPay701', 'KoiPayment701-微信H5', '华安支付701-收银台',
       '工-吴娅楠-VIP2~VIP10_極速', '工-曹翠翠-VIP2~VIP10_極速',
       '建-陈柏宗-VIP2~VIP10_極速', '拉斐支付707-微信H5', '新快讯703-支付宝H5',
       '新快讯704-微信H5', '新快讯705-支付宝H5', '新快讯706-收银台', '新龙腾703-收银台',
       '永信701-收银台', '牛公公706-收银台', '牛公公707-收银台', '牛公公726-银联H5',
       '码上支付701-支付宝QR', '码上支付705-微信QR', '购宝钱包701', '远华支付701-收银台',
       '远华支付702-银联H5'], dtype=object)

In [111]:
emptylist= pd.DataFrame(columns=['Resource','Start','Finish'])

for j in name_list :
    test_data=df_data[df_data['alias']==j].sort_index(axis=0, ascending=False)
    data_len=test_data.shape[0]
    for i in range(data_len):
        if test_data['status'].iloc[0]=='disable' or test_data['status'].iloc[0]=='suspend':
            if i == 0 :
                emptylist=emptylist.append({"Resource":test_data['alias'].iloc[0],"Start":aaaaa1+'T00:00:00+0800', "Finish":test_data['created_at'].iloc[0]},ignore_index=True)
            if test_data['status'].iloc[i]=='disable' and test_data['status'].iloc[i]=='suspend':
                continue
            if test_data['status'].iloc[i] == 'enable' and i != data_len-1: 
                emptylist=emptylist.append({"Resource":test_data['alias'].iloc[0],"Start":test_data['created_at'].iloc[i], "Finish":test_data['created_at'].iloc[i+1]},ignore_index=True)
            if i == data_len-1 and test_data['status'].iloc[i] == 'enable':
                emptylist=emptylist.append({"Resource":test_data['alias'].iloc[0],"Start":test_data['created_at'].iloc[i], "Finish":aaaaa2+'T11:59:59+0800'},ignore_index=True)

    
        if test_data['status'].iloc[0]=='enable':
            if i == 0 and i != data_len-1:
                emptylist=emptylist.append({"Resource":test_data['alias'].iloc[0],"Start":test_data['created_at'].iloc[i], "Finish":test_data['created_at'].iloc[i+1]},ignore_index=True)
            if test_data['status'].iloc[i]=='disable' and test_data['status'].iloc[i]=='suspend':
                continue
            if test_data['status'].iloc[i] == 'enable' and i != data_len-1 and i != 0: 
                emptylist=emptylist.append({"Resource":test_data['alias'].iloc[0],"Start":test_data['created_at'].iloc[i], "Finish":test_data['created_at'].iloc[i+1]},ignore_index=True)
            if i == data_len-1 and test_data['status'].iloc[i] == 'enable':
                emptylist=emptylist.append({"Resource":test_data['alias'].iloc[0],"Start":test_data['created_at'].iloc[i], "Finish":aaaaa2+'T11:59:59+0800'},ignore_index=True)


In [112]:
emptylist['time_open'] = pd.to_datetime(emptylist['Finish'])-pd.to_datetime(emptylist['Start'])

In [113]:
emptylist['Finish']=(pd.to_datetime(emptylist['Finish'])- datetime.timedelta(hours=12)).dt.strftime("%Y-%m-%d %H:%M:%S") 
emptylist['Start']=(pd.to_datetime(emptylist['Start'])- datetime.timedelta(hours=12)).dt.strftime("%Y-%m-%d %H:%M:%S") 

In [115]:
final_data=pd.DataFrame(emptylist.groupby(by=['Resource']).agg({'time_open': ['count','sum']})).reset_index().rename(columns={ "count": "total_count","sum": "total_time"})
final_data.columns = ['Resource', 'total_count', 'total_time']

In [116]:
#合併通道金額大小
df_min_max=pd.DataFrame(ttt222.json()['ret'])[['alias','per_trade_min','per_trade_max']]
df_min_max.columns =  ['Resource', 'per_trade_min', 'per_trade_max'] 
df_finaldata=pd.merge(all_channel,final_data,on = 'Resource',how = 'outer')
df_finaldata=pd.merge(df_finaldata,df_min_max,on = 'Resource',how = 'left')

In [117]:
df_finaldata['total_count']=df_finaldata['total_count'].fillna(1)
df_finaldata['total_time']=df_finaldata['total_time'].fillna('1 days 12:00:00')
df_finaldata=df_finaldata.fillna('銀行卡系列')

In [118]:
#list_data_all_s list_data_all_f
df_data_s=pd.DataFrame(list_data_all_s)[['merchant_alias','user_id','level_name','confirm','amount']]
df_data_f=pd.DataFrame(list_data_all_f)[['merchant_alias','user_id','level_name','confirm']]

#排除測試和代客
df_data_s=df_data_s[(df_data_s['merchant_alias'] != '代客充值-鸭脖银行卡支援') & (df_data_s['level_name'] != '測試帳號') & (df_data_s['confirm'] == True)]
df_data_f=df_data_f[(df_data_f['merchant_alias'] != '代客充值-鸭脖银行卡支援') & (df_data_f['level_name'] != '測試帳號') & (df_data_f['confirm'] == False)] 
df_data_s['amount'] = df_data_s['amount'].astype('float').astype('int')

#groupby 
df_data_s_count=pd.DataFrame(df_data_s.groupby(by=['merchant_alias']).agg({'amount': ['count','sum']})).rename(columns={ "user_id": "success_total"}).reset_index()
df_data_f_count=pd.DataFrame(df_data_f.groupby(by=['merchant_alias'])['user_id'].count()).rename(columns={ "user_id": "fail_total"}).reset_index()
df_data_s_people=pd.DataFrame(df_data_s.groupby(by=['merchant_alias'])['user_id'].nunique()).rename(columns={ "user_id": "success_user"}).reset_index()
df_data_s_count.columns = ['Resource','success_total','success_amount']
df_data_f_count.columns = ['Resource','fail_total']
df_data_s_people.columns = ['Resource','success_user']

#merge
all_resource_df=pd.merge(df_finaldata,df_data_s_count,on = 'Resource',how = 'left')
all_resource_df=pd.merge(all_resource_df,df_data_f_count,on = 'Resource',how = 'left')
all_resource_df=all_resource_df.fillna(0)
all_resource_df['total_s_f']=(all_resource_df['success_total']+all_resource_df['fail_total'])
all_resource_df['success_rate']=round(all_resource_df['success_total'] / all_resource_df['total_s_f'] *100,1)
all_resource_df['fail_rate']=round(all_resource_df['fail_total'] / all_resource_df['total_s_f'] *100,1)
all_resource_df=pd.merge(all_resource_df,df_data_s_people,on = 'Resource',how = 'left')

#remove na
all_resource_df=all_resource_df.fillna(0)

In [131]:
aaaaa1_1=(datetime.datetime.now() - datetime.timedelta(hours=48)).strftime("%Y-%m-%d") 
column_add=pd.DataFrame(df_finaldata[df_finaldata['total_time']=='1 days 12:00:00']['Resource'])
column_add['Start']=aaaaa1_1+' 12:00:00'  #00
column_add['Finish']=aaaaa1+' 23:59:59'  #23 aaaaa
column_add['time_open']='1 days 12:00:00'
testdata_all=emptylist.append(column_add)

In [132]:
testdata_all=pd.merge(testdata_all,df_min_max,on = 'Resource',how = 'left')
testdata_all=pd.merge(testdata_all,all_channel,on = 'Resource',how = 'left')
testdata_all=testdata_all.fillna(0) # pd.to_numeric(testdata_all['per_trade_min'], errors='ignore').astype(str)
testdata_all['per_trade_min']=pd.to_numeric(testdata_all['per_trade_min'], downcast='integer')
testdata_all['Resource_name']=testdata_all['Resource']+'-'+testdata_all['per_trade_min'].astype(str)
testdata_all.sort_values(by=["per_trade_min",'bank_name'],inplace=True)

In [133]:
import plotly.express as px
import pandas as pd

fig = px.timeline(testdata_all, x_start="Start", x_end="Finish", y="Resource_name", color="Resource" ,title='Channel Timeline Eastern')
#fig.show()

In [134]:
#存取到本地
import plotly
import plotly.graph_objs as go
plotly.offline.plot(fig, filename = 'F:/Desktop/download_csv/filename_eastern.html', auto_open=False)
all_resource_df.to_csv('F:/Desktop/download_csv/channel_data_eastern.csv' ,encoding="utf_8_sig" )

In [28]:
#傳送到 TG
import telepot
tele_chatid=['-451149494']         #测试 -451149494   #正式 -空 
tele_token='1020859504:AAEb-tLbaBjJvJqBsLCzCsStrgTlZNqXRR8'
bot = telepot.Bot(tele_token)
bot.sendMessage(chat_id=tele_chatid[0],text= '美東時間 : '+ str(aaaaa1) +"\n" ) 
bot.sendDocument(chat_id=tele_chatid[0] , document= open('F:/Desktop/download_csv/filename_eastern.html','rb')) 
bot.sendDocument(chat_id=tele_chatid[0] , document= open('F:/Desktop/download_csv/channel_data_eastern.csv','rb'))#,encoding = 'utf-8'

{'message_id': 4104,
 'from': {'id': 1020859504,
  'is_bot': True,
  'first_name': 'BI_Bot',
  'username': 'torin_gcp_bot'},
 'chat': {'id': -451149494,
  'title': 'BI-team',
  'type': 'group',
  'all_members_are_administrators': True},
 'date': 1609305030,
 'document': {'file_name': 'channel_data.csv',
  'mime_type': 'text/csv',
  'file_id': 'BQACAgUAAxkDAAIQCF_sC8YcQ_Nq-SlAtNG7ylwkCqy8AAKNAQAC201gV60cfTA6n0mjHgQ',
  'file_unique_id': 'AgADjQEAAttNYFc',
  'file_size': 2796}}

In [135]:
#傳送到 TG
import telepot
tele_chatid=['-408461960']         #测试 -451149494   #正式 -空 
tele_token='1020859504:AAEb-tLbaBjJvJqBsLCzCsStrgTlZNqXRR8'
bot = telepot.Bot(tele_token)
bot.sendMessage(chat_id=tele_chatid[0],text= '美東時間 : '+ str(aaaaa1) +"\n" ) 
bot.sendDocument(chat_id=tele_chatid[0] , document= open('F:/Desktop/download_csv/filename_eastern.html','rb')) 
bot.sendDocument(chat_id=tele_chatid[0] , document= open('F:/Desktop/download_csv/channel_data_eastern.csv','rb'))#,encoding = 'utf-8'

{'message_id': 4305,
 'from': {'id': 1020859504,
  'is_bot': True,
  'first_name': 'BI_Bot',
  'username': 'torin_gcp_bot'},
 'chat': {'id': -408461960,
  'title': 'H-數據通道群',
  'type': 'group',
  'all_members_are_administrators': True},
 'date': 1609917256,
 'document': {'file_name': 'channel_data_eastern.csv',
  'mime_type': 'text/csv',
  'file_id': 'BQACAgUAAxkDAAIQ0V_1Y0gGCPRA6-x52eIOIelt6lrzAALCAQACBmyxVxCpn-QlM2MIHgQ',
  'file_unique_id': 'AgADwgEAAgZssVc',
  'file_size': 3422}}

In [ ]:
import os 
os.system("taskkill /im firefox.exe")

In [4]:
'''
test_data_123=df_data[df_data['alias']=="KoiPayment701-微信H5"].sort_index(axis=0, ascending=False)
test_data_123

def send_telegrame():
    tele_chatid=['-408673375']         #测试 -451149494   #正式 -空 
    tele_token='1020859504:AAEb-tLbaBjJvJqBsLCzCsStrgTlZNqXRR8'
    bot = telepot.Bot(tele_token)
    bot.sendMessage(chat_id=tele_chatid[0],
                text= '美東時間 : '+ str(delay_oneday) +'時'+ "\n" +
                '註冊人數 : '+ str(player) + "\n" +
                '有效投注金額 : '+ str(valid_bet) + "\n" +
                '損益 : '+ str(payoff) + "\n" +
                '首存人數 : '+ str(deposit_first) + "\n" +
                '存款總額 : '+ str(deposit) + "\n" +
                '取款總額 : '+ str(withdraw) + "\n" +
                '獲利比(損益/有效投注) : '+ str(profit_ratio) +'%'+ "\n" )
#獲取訊息
player=int(ttt.json()['ret']['player']['today'])
valid_bet=float(ttt.json()['ret']['valid_bet']['today'])
payoff=float(ttt.json()['ret']['payoff'] ['today'])*-1
deposit_first=int(ttt.json()['ret']['deposit_first'] ['today'])
deposit=float(ttt.json()['ret']['deposit'] ['today'])
withdraw=float(ttt.json()['ret']['withdraw'] ['today'])
profit_ratio=round(float(ttt.json()['ret']['profit_ratio'] ['today']),2)
#傳送到TG
send_telegrame()    
df = pd.DataFrame([
    dict(Task="Job A", Start='2020-12-21T00:02:04+0800 ', Finish='2020-12-21T01:07:24+0800', Resource="新快讯704-微信H5"),
    dict(Task="Job B", Start='2020-12-21T09:33:07+0800', Finish='2020-12-21T16:43:21+0800', Resource="新快讯704-微信H5"),
    dict(Task="Job C", Start='2020-12-21T04:33:07+0800', Finish='2020-12-21T12:43:21+0800', Resource="牛公公706-收银台")
])

fig.update_layout(xaxis=dict(
                      title='Timestamp', 
                      tickformat = '%H:%M:%S',
                  ))


'''

In [26]:
trydata=pd.merge(emptylist,df_min_max,on = 'Resource',how = 'left')

trydata_50=trydata[trydata['per_trade_min']=='50.0000']
trydata_50.sort_values(by=["Start"],inplace=True)
trydata_50['Start']=pd.to_datetime(trydata_50['Start'])
trydata_50['Finish']=pd.to_datetime(trydata_50['Finish'])

trydata_100=trydata[trydata['per_trade_min']=='100.0000']
trydata_100.sort_values(by=["Start"],inplace=True)
trydata_100['Start']=pd.to_datetime(trydata_100['Start'])
trydata_100['Finish']=pd.to_datetime(trydata_100['Finish'])

<ipython-input-26-21cb78e33c94>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trydata_50.sort_values(by=["Start"],inplace=True)
<ipython-input-26-21cb78e33c94>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trydata_50['Start']=pd.to_datetime(trydata_50['Start'])
<ipython-input-26-21cb78e33c94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [35]:
def find_small_time(data,text,logistic='f'):
    list_resource_50= pd.DataFrame(columns=['Resource','Resource_name','Start','Finish'])
    j=0
    if data['Start'].iloc[0] != pd.to_datetime(aaaaa1+' 00:00:00') :
        list_resource_50=list_resource_50.append({"Resource":text,"Resource_name":text,"Start":aaaaa1+' 00:00:00', "Finish":data['Start'].iloc[0]},ignore_index=True)  
    if data['Finish'].iloc[data.shape[0]-1] != pd.to_datetime(aaaaa1+' 23:59:59') :
        list_resource_50=list_resource_50.append({"Resource":text,"Resource_name":text,"Start":data['Finish'].iloc[data.shape[0]-1], "Finish":aaaaa1+' 23:59:59'},ignore_index=True)  
    
    for i in range(data.shape[0]):
        range_time=(data['Finish'].iloc[j]-data['Start'].iloc[j]).total_seconds()
        if (data['Start'].iloc[i]-data['Start'].iloc[j]).total_seconds() < range_time :
            logistic='t'
            continue
            
        list_resource_50=list_resource_50.append({"Resource":text,"Resource_name":text,"Start":data['Finish'].iloc[j], "Finish":data['Start'].iloc[i]},ignore_index=True)  
        if logistic=='t' :
            j=i
            logistic='f'    
            
    return(list_resource_50)

In [36]:
no_50_time=find_small_time(trydata_50,'小額50未使用區間')
no_100_time=find_small_time(trydata_100,'小額100未使用區間')

In [31]:
plotly_data=testdata_all[['Resource','Resource_name','Start','Finish']].append(no_50_time).append(no_100_time)

In [250]:
#pd.to_datetime(testdata_all['Start']).dt.strftime("%Y-%m-%d %H:%M:%S") 


In [30]:
trydata_50

,Resource,Start,Finish,time_open,per_trade_min,per_trade_max
0,KoiPayment701-微信H5,2021-01-03 00:00:00,2021-01-03 11:43:05,0 days 11:43:05,50.0000,200.0000
19,拉斐支付707-微信H5,2021-01-03 00:00:00,2021-01-03 08:11:48,0 days 08:11:48,50.0000,200.0000
24,新快讯704-微信H5,2021-01-03 00:00:00,2021-01-03 07:41:09,0 days 07:41:09,50.0000,200.0000
38,远华支付702-银联H5,2021-01-03 04:59:26,2021-01-03 23:59:59,0 days 19:00:33,50.0000,20000.0000
28,新快讯706-收银台,2021-01-03 12:22:35,2021-01-03 18:56:41,0 days 06:34:06,50.0000,1499.0000
1,KoiPayment701-微信H5,2021-01-03 20:43:17,2021-01-03 23:59:59,0 days 03:16:42,50.0000,200.0000
20,拉斐支付707-微信H5,2021-01-03 21:18:31,2021-01-03 23:59:59,0 days 02:41:28,50.0000,200.0000
25,新快讯704-微信H5,2021-01-03 23:48:13,2021-01-03 23:59:59,0 days 00:11:46,50.0000,200.0000


In [37]:
no_50_time

,Resource,Resource_name,Start,Finish
0,小額50未使用區間,小額50未使用區間,2021-01-03 11:43:05,2021-01-03 12:22:35
1,小額50未使用區間,小額50未使用區間,2021-01-03 18:56:41,2021-01-03 20:43:17
2,小額50未使用區間,小額50未使用區間,2021-01-03 18:56:41,2021-01-03 21:18:31
3,小額50未使用區間,小額50未使用區間,2021-01-03 18:56:41,2021-01-03 23:48:13
